In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import os
# path = ""
# path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/'
# os.chdir(path)
# import time
# FOLDERNAME = "OPUS-OpenNMT-sent2vec-0.85-" + str(time.strftime("%Y%m%d-%H%M"))
# !mkdir $FOLDERNAME

# path = path + FOLDERNAME
# os.chdir(path)
# !pwd

# import os
# path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/OPUS-OpenNMT-sent2vec-0.85-20210412-2246'
# os.chdir(path)
# !pwd

In [3]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## **Install libraries**

In [4]:
!pip install OpenNMT-py==1.2.0
!pip install -U scikit-learn

     |████████████████████████████████| 204kB 14.6MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 61kB 5.3MB/s 
     |████████████████████████████████| 14.3MB 13.7MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=d1ae150cc6007caab629b69a5fabf1c6d9df6a98eea7081c8ff0a675f6edbbbd
  Stored in directory: /root/.cache/pip/wheels/d6/61/f7/626bbd080a9f2f70015f92025e0af663c595146083f3d9aa05
Successfully built configargparse
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
     |████████████████████████████████| 22.3MB 1.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [5]:
!wget https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/opus_sent2vec.tar.gz
!mkdir data_bin
!tar -xvf 'opus_sent2vec.tar.gz'

--2021-04-28 04:17:29--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/opus_sent2vec.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94255466 (90M) [application/octet-stream]
Saving to: ‘opus_sent2vec.tar.gz’

opus_sent2vec.tar.g 100%[===================>]  89.89M   268MB/s    in 0.3s    

2021-04-28 04:17:30 (268 MB/s) - ‘opus_sent2vec.tar.gz’ saved [94255466/94255466]

en_train_EM_score_0.95
vi_valid
en_train_EM_0.95
en_train_EM_factor_0.85
en_train_EM_score_0.8
vi_train
en_train_EM_factor_0.8
en_train_EM_0.8
en_valid
en_train_EM_factor_0.95
en_train
en_train_EM_score_0.85
vi_test
en_train_EM_0.85
en_train_EM_score_0.9
en_test
en_train_EM_factor_0.9
en_train_EM_0.9


In [6]:
!mkdir -p output
!onmt_preprocess -train_src 'en_train_EM_0.8' \\
-train_tgt 'vi_train' \\
-valid_src 'en_valid' \\
-valid_tgt 'vi_valid' \\
-save_data 'output/en-vi' 

[2021-04-28 04:17:36,778 INFO] Extracting features...
[2021-04-28 04:17:36,778 INFO]  * number of source features: 0.
[2021-04-28 04:17:36,778 INFO]  * number of target features: 0.
[2021-04-28 04:17:36,778 INFO] Building `Fields` object...
[2021-04-28 04:17:36,778 INFO] Building & saving training data...
[2021-04-28 04:17:38,119 INFO] Building shard 0.
[2021-04-28 04:18:11,382 INFO]  * saving 0th train data shard to output/en-vi.train.0.pt.
[2021-04-28 04:18:30,925 INFO]  * tgt vocab size: 50004.
[2021-04-28 04:18:31,369 INFO]  * src vocab size: 50002.
[2021-04-28 04:18:32,135 INFO] Building & saving validation data...
[2021-04-28 04:18:32,849 INFO] Building shard 0.
[2021-04-28 04:18:35,198 INFO]  * saving 0th valid data shard to output/en-vi.valid.0.pt.


In [7]:
!mkdir -p model
!onmt_train -data 'output/en-vi' \\
-save_model 'model/en-vi' \\
-layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 \\
-encoder_type transformer -decoder_type transformer -position_encoding \\
-train_steps 30000  -max_generator_batches 2 -dropout 0.1 -batch_size 4096 \\
-batch_type tokens -normalization tokens  -accum_count 2 -optim adam -adam_beta2 0.998 \\
-decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 \\
-param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 \\
-report_every 1000 -world_size 1 -gpu_ranks 0

Traceback (most recent call last):
  File "/usr/local/bin/onmt_train", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/train.py", line 197, in main
    train(opt)
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/train.py", line 95, in train
    single_main(opt, 0)
  File "/usr/local/lib/python3.7/dist-packages/onmt/train_single.py", line 45, in main
    configure_process(opt, device_id)
  File "/usr/local/lib/python3.7/dist-packages/onmt/train_single.py", line 38, in configure_process
    torch.cuda.set_device(device_id)
  File "/usr/local/lib/python3.7/dist-packages/torch/cuda/__init__.py", line 261, in set_device
    torch._C._cuda_setDevice(device)
  File "/usr/local/lib/python3.7/dist-packages/torch/cuda/__init__.py", line 170, in _lazy_init
    torch._C._cuda_init()
RuntimeError: No CUDA GPUs are available


In [8]:
!ls -al model model/

model:
total 8
drwxr-xr-x 2 root root 4096 Apr 28 04:18 .
drwxr-xr-x 1 root root 4096 Apr 28 04:18 ..

model/:
total 8
drwxr-xr-x 2 root root 4096 Apr 28 04:18 .
drwxr-xr-x 1 root root 4096 Apr 28 04:18 ..


In [9]:
!onmt_translate -model model/en-vi_step_30000.pt -src en_test -tgt vi_test -output predict.txt

Traceback (most recent call last):
  File "/usr/local/bin/onmt_translate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/translate.py", line 48, in main
    translate(opt)
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/translate.py", line 18, in translate
    translator = build_translator(opt, logger=logger, report_score=True)
  File "/usr/local/lib/python3.7/dist-packages/onmt/translate/translator.py", line 28, in build_translator
    fields, model, model_opt = load_test_model(opt)
  File "/usr/local/lib/python3.7/dist-packages/onmt/model_builder.py", line 95, in load_test_model
    map_location=lambda storage, loc: storage)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 579, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/li

In [10]:
!tail vi_test

And nobody questions him, because they don't want to hear the answer because it's a lie!
Kubo?
Họ rất vui vẻ, và lúc nào cũng hát với nến.
Nghe này, anh không thể nói chuyện bây giờ được.
Vậy thì con có thể dùng trí tưởng tượng của mình.
Không hề.
Tôi đang nhìn hắn ngay lúc này đây.
Bác không để tâm chứ?
Anh nghĩ cậu ta phản ứng với thuốc?
Bị làm sao mà anh lại đi dự lễ đặt tên em bé của Cuddy chứ?


In [11]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 17287, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 17287 (delta 146), reused 132 (delta 76), pack-reused 17044
Receiving objects: 100% (17287/17287), 273.53 MiB | 30.89 MiB/s, done.
Resolving deltas: 100% (12446/12446), done.


In [12]:
!ls -al

total 351740
drwxr-xr-x  1 root root     4096 Apr 28 04:18 .
drwxr-xr-x  1 root root     4096 Apr 28 04:14 ..
drwxr-xr-x  4 root root     4096 Apr 21 13:38 .config
drwxr-xr-x  2 root root     4096 Apr 28 04:17 data_bin
drwx------  6 root root     4096 Apr 28 04:17 drive
-rw-rw-r--  1 1000 1000  3318349 Apr 12 02:53 en_test
-rw-rw-r--  1 1000 1000 26563375 Apr 12 02:53 en_train
-rw-rw-r--  1 1000 1000 31608088 Apr 12 03:46 en_train_EM_0.8
-rw-rw-r--  1 1000 1000 29845474 Apr 12 03:46 en_train_EM_0.85
-rw-rw-r--  1 1000 1000 28377219 Apr 12 03:46 en_train_EM_0.9
-rw-rw-r--  1 1000 1000 27245135 Apr 12 03:46 en_train_EM_0.95
-rw-rw-r--  1 1000 1000 11930822 Apr 12 03:46 en_train_EM_factor_0.8
-rw-rw-r--  1 1000 1000 11270762 Apr 12 03:46 en_train_EM_factor_0.85
-rw-rw-r--  1 1000 1000 10719696 Apr 12 03:46 en_train_EM_factor_0.9
-rw-rw-r--  1 1000 1000 10293086 Apr 12 03:46 en_train_EM_factor_0.95
-rw-rw-r--  1 1000 1000  6897191 Apr 12 03:46 en_train_EM_score_0.8
-rw-rw-r--  1 1000 1000 

In [13]:
!perl OpenNMT-py/tools/multi-bleu.perl vi_test < predict.txt

Use of uninitialized value $length_reference in numeric eq (==) at OpenNMT-py/tools/multi-bleu.perl line 148.
BLEU = 0, 0/0/0/0 (BP=0, ratio=0, hyp_len=0, ref_len=0)
